In [20]:
# ## Cell 1: Import Libraries and Define Paths
#
# In this cell, we import the necessary libraries and define the file paths for the dataset, BioWordVec embeddings, and output files.

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import os
from gensim.models import KeyedVectors

# Paths to files (adjust as needed)
DATA_PATH = "DiseaseAndSymptoms.csv"
BIOWORDVEC_PATH = r"C:\Users\ACER\Downloads\BioWordVec_PubMed_MIMICIII_d200.vec.bin" # Path to BioWordVec embeddings in .bin format
MODEL_PATH = "biowordvec_diagnosis_model.keras"
TOKENIZER_PATH = "symptom_tokenizer.pkl"
LABEL_ENCODER_PATH = "label_encoder.pkl"

In [15]:
# ## Cell 2: Load and Preprocess the Dataset
#
# This cell loads the dataset (`DiseaseAndSymptoms.csv`), preprocesses it, and splits it into train/test sets.

def load_and_preprocess_data(data_path):
    """
    Load the dataset, preprocess symptoms, and split into train/test sets.
    """
    data = pd.read_csv(data_path)
    data["Disease"] = data["Disease"].replace("Peptic ulcer diseae", "Peptic ulcer disease")
    data["Disease"] = data["Disease"].replace("Dimorphic hemmorhoids(piles)", "Dimorphic hemorrhoids (piles)")
    
    # Replace underscores with spaces in column names and data
    data.columns = [col.replace("_", " ") for col in data.columns]
    data = data.apply(lambda x: x.str.replace("_", " ") if x.dtype == "object" else x)
    
    # Combine all symptoms into a single string per row
    symptom_cols = [col for col in data.columns if "Symptom" in col]
    data["Symptoms"] = data[symptom_cols].apply(
        lambda row: " ".join(sorted(set([s.strip() for s in row if pd.notna(s)]))), axis=1
    )
    
    # Drop rows with empty symptoms
    data = data[data["Symptoms"].str.strip() != ""]
    
    # Split into train and test sets (before augmentation)
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    
    return train_data[["Disease", "Symptoms"]], test_data[["Disease", "Symptoms"]]

print("Loading and preprocessing data...")
train_data, test_data = load_and_preprocess_data(DATA_PATH)
print(f"Training dataset size: {len(train_data)}, Test dataset size: {len(test_data)}")

Loading and preprocessing data...
Training dataset size: 3936, Test dataset size: 984


In [16]:
# ## Cell 3: Augment the Training Dataset
#
# This cell augments the training dataset by creating synthetic symptom combinations for each disease.

def augment_data(data, samples_per_disease=20):
    """
    Augment the dataset by creating synthetic symptom combinations with co-occurrence patterns.
    """
    augmented_data = data.copy()
    diseases = data["Disease"].unique()
    
    # For each disease, create synthetic samples
    for disease in diseases:
        disease_rows = data[data["Disease"] == disease]
        all_symptoms = set()
        for symptoms in disease_rows["Symptoms"]:
            all_symptoms.update(symptoms.split())
        all_symptoms = list(all_symptoms)
        
        # Generate synthetic samples
        for _ in range(samples_per_disease):
            # Randomly select 2 to min(len(all_symptoms), 5) symptoms
            num_symptoms = np.random.randint(2, min(len(all_symptoms) + 1, 6))
            selected_symptoms = np.random.choice(all_symptoms, size=num_symptoms, replace=False)
            synthetic_symptoms = " ".join(sorted(selected_symptoms))
            augmented_data = pd.concat([augmented_data, pd.DataFrame({
                "Disease": [disease],
                "Symptoms": [synthetic_symptoms]
            })], ignore_index=True)
    
    return augmented_data

print("Augmenting training data...")
augmented_train_data = augment_data(train_data, samples_per_disease=20)
print(f"Original training dataset size: {len(train_data)}, Augmented training dataset size: {len(augmented_train_data)}")

Augmenting training data...
Original training dataset size: 3936, Augmented training dataset size: 4756


In [18]:
# ## Cell 4: Prepare Data for Training
#
# This cell tokenizes the symptoms, encodes the diseases, and prepares the training, validation, and test sets.

def prepare_data(train_data, test_data, max_len=20):
    """
    Tokenize symptoms, encode diseases, and prepare training, validation, and test data.
    """
    # Tokenize symptoms
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_data["Symptoms"])
    
    # Convert symptoms to sequences
    train_sequences = tokenizer.texts_to_sequences(train_data["Symptoms"])
    train_sequences = pad_sequences(train_sequences, maxlen=max_len, padding="post")
    
    test_sequences = tokenizer.texts_to_sequences(test_data["Symptoms"])
    test_sequences = pad_sequences(test_sequences, maxlen=max_len, padding="post")
    
    # Encode diseases
    label_encoder = LabelEncoder()
    train_labels = label_encoder.fit_transform(train_data["Disease"])
    test_labels = label_encoder.transform(test_data["Disease"])
    
    # Split augmented training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        train_sequences, train_labels, test_size=0.2, random_state=42
    )
    
    # Compute class weights to handle class imbalance
    class_weights = compute_class_weight("balanced", classes=np.unique(train_labels), y=train_labels)
    class_weight_dict = dict(enumerate(class_weights))
    
    return X_train, X_val, y_train, y_val, test_sequences, test_labels, tokenizer, label_encoder, len(tokenizer.word_index) + 1, class_weight_dict

print("Preparing data for training...")
X_train, X_val, y_train, y_val, X_test, y_test, tokenizer, label_encoder, vocab_size, class_weight_dict = prepare_data(augmented_train_data, test_data)
num_classes = len(label_encoder.classes_)
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}, Test samples: {len(X_test)}")
print(f"Vocabulary size: {vocab_size}, Number of classes: {num_classes}")

Preparing data for training...
Training samples: 3804, Validation samples: 952, Test samples: 984
Vocabulary size: 209, Number of classes: 41


In [19]:
# ## Cell 5: Load BioWordVec Embeddings
#
# This cell loads the BioWordVec embeddings from the .bin file using gensim's KeyedVectors.

def load_biowordvec_embeddings(biowordvec_path):
    """
    Load BioWordVec embeddings from a .bin file using gensim.
    """
    print("Loading BioWordVec embeddings from .bin file...")
    embeddings = KeyedVectors.load_word2vec_format(biowordvec_path, binary=True)
    return embeddings

print("Loading BioWordVec embeddings...")
embeddings_index = load_biowordvec_embeddings(BIOWORDVEC_PATH)
print(f"Loaded {len(embeddings_index)} word vectors.")

Loading BioWordVec embeddings...
Loading BioWordVec embeddings from .bin file...
Loaded 16545452 word vectors.


In [21]:
# ## Cell 6: Create Embedding Matrix
#
# This cell creates an embedding matrix using the BioWordVec embeddings for the tokenized vocabulary.

def create_embedding_matrix(tokenizer, embeddings_index, vocab_size, embedding_dim=200):
    """
    Create an embedding matrix for the model using BioWordVec embeddings.
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if i >= vocab_size:
            continue
        try:
            embedding_vector = embeddings_index[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            # Word not in BioWordVec vocabulary, leave as zero vector
            pass
    return embedding_matrix

print("Creating embedding matrix...")
embedding_matrix = create_embedding_matrix(tokenizer, embeddings_index, vocab_size)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Creating embedding matrix...
Embedding matrix shape: (209, 200)


In [22]:
# ## Cell 7: Build and Train the Model
#
# This cell builds a simpler dense model with BioWordVec embeddings and trains it on the augmented dataset.

def build_and_train_model(X_train, X_val, y_train, y_val, vocab_size, embedding_matrix, num_classes, class_weight_dict, max_len=20, embedding_dim=200):
    """
    Build and train a simpler dense model with BioWordVec embeddings.
    """
    model = Sequential([
        Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=True),  # Fine-tune embeddings
        Flatten(),
        Dense(64, activation="relu"),
        Dropout(0.5),
        Dense(32, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    model.summary()
    
    # Add early stopping
    early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    
    # Train the model with class weights
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        callbacks=[early_stopping],
        class_weight=class_weight_dict,
        verbose=1
    )
    
    return model, history

print("Building and training model...")
model, history = build_and_train_model(
    X_train, X_val, y_train, y_val, vocab_size, embedding_matrix, num_classes, class_weight_dict
)

Building and training model...


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │        41,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,800 (163.28 KB)

 Trainable params: 41,800 (163.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0936 - loss: 3.4105 - val_accuracy: 0.7584 - val_loss: 1.6513
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3310 - loss: 2.3976 - val_accuracy: 0.8414 - val_loss: 1.0451
Epoch 3/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4178 - loss: 2.0330 - val_accuracy: 0.8739 - val_loss: 0.7952
Epoch 4/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4889 - loss: 1.7868 - val_accuracy: 0.8887 - val_loss: 0.6796
Epoch 5/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5296 - loss: 1.6077 - val_accuracy: 0.8992 - val_loss: 0.6008
Epoch 6/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5467 - loss: 1.5591 - val_accuracy: 0.8971 - val_loss: 0.5459
Epoch 7/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6002 - loss: 1.3401 - val_accuracy: 0.9034 - val_loss: 0.4943
Epoch 8/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.6014 - loss: 1.3209 - val_accuracy: 0

In [23]:
# ## Cell 8: Evaluate on Test Set
#
# This cell evaluates the model on the separate test set.

print("Evaluating on test set...")
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Evaluating on test set...
Test Loss: 0.0118, Test Accuracy: 1.0000


In [24]:
# ## Cell 9: Save the Model, Tokenizer, and Label Encoder
#
# This cell saves the trained model, tokenizer, and label encoder for use in the inference script (`main.py`).

print("Saving model, tokenizer, and label encoder...")
model.save(MODEL_PATH)
with open(TOKENIZER_PATH, "wb") as f:
    pickle.dump(tokenizer, f)
with open(LABEL_ENCODER_PATH, "wb") as f:
    pickle.dump(label_encoder, f)

print("Training complete!")

Saving model, tokenizer, and label encoder...
Training complete!
